In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": { "scrolled": true },
   "outputs": [],
   "source": [
    "%cd /content/\n",
    "# ==============================================================\n",
    "# 1) WAN22 + ENV + TUTORIAL IMAGE + SOUND + COMFYUI\n",
    "# ==============================================================\n",
    "!wget https://huggingface.co/banhkeomath1/and/resolve/main/tutorial.png -P /content/ -q\n",
    "import base64, os, subprocess, threading, time, socket, re\n",
    "from IPython.display import Audio, Javascript, HTML, Image, display\n",
    "\n",
    "with open('/content/tutorial.png','rb') as f:\n",
    "    html_img = 'data:image/png;base64,' + base64.b64encode(f.read()).decode()\n",
    "print('✔ Tutorial encoded')\n",
    "\n",
    "!wget https://huggingface.co/banhkeomath2/wan22/resolve/main/wan22.sh -q\n",
    "!wget https://huggingface.co/banhkeomath2/wan22/resolve/main/env.txt -q\n",
    "!apt -y install aria2\n",
    "!git clone https://github.com/comfyanonymous/ComfyUI.git\n",
    "from google.colab import drive\n",
    "drive.mount('/content/drive')\n",
    "\n",
    "%cd /content/ComfyUI\n",
    "env_file = '/content/env.txt'\n",
    "with open(env_file,'r') as f:\n",
    "    for line in f:\n",
    "        if '=' in line:\n",
    "            k,v = line.strip().split('=',1)\n",
    "            os.environ[k] = v\n",
    "\n",
    "dirs = [\n",
    " os.environ.get('dif'),\n",
    " os.environ.get('cp'),\n",
    " os.environ.get('loras/wan22'),\n",
    " os.environ.get('clip'),\n",
    " os.environ.get('clipv'),\n",
    " os.environ.get('lorasf'),\n",
    " os.environ.get('ipadapter'),\n",
    " os.environ.get('loras15xl'),\n",
    " os.environ.get('cnt'),\n",
    " os.environ.get('birefnet'),\n",
    " os.environ.get('upscale'),\n",
    " os.environ.get('vae')\n",
    "]\n",
    "for d in dirs:\n",
    "    if d:\n",
    "        os.makedirs(d, exist_ok=True)\n",
    "        print('Created:', d)\n",
    "\n",
    "!hf download banhkeomath2/sound --local-dir /content/sound/\n",
    "audio_path = '/content/sound/1.mp3'\n",
    "\n",
    "def play_audio(p):\n",
    "    display(Audio(p, autoplay=True))\n",
    "    display(Javascript('document.querySelector(\"audio\").style.display=\"none\"'))\n",
    "\n",
    "play_audio(audio_path)\n",
    "\n",
    "# ==============================================================\n",
    "# 2) PINGGY TUNNEL + COMFYUI AUTO START\n",
    "# ==============================================================\n",
    "%cd /content/ComfyUI\n",
    "!pkill -f \"python.*main.py\"\n",
    "!pkill -f \"ssh.*pinggy\"\n",
    "\n",
    "COMFYUI_PORT = 8188\n",
    "PINGGY_LOCAL_PORT = '4300:localhost:4300'\n",
    "PINGGY_TOKEN = 'free.pinggy.io'\n",
    "\n",
    "tunnel_proc = None\n",
    "tunnel_url = None\n",
    "tunnel_ready = False\n",
    "\n",
    "def extract_tunnel_url(text):\n",
    "    patterns = [\n",
    "        r'https://[a-zA-Z0-9\\-]+\\.a\\.free\\.pinggy\\.link',\n",
    "        r'https://[a-zA-Z0-9\\-]+\\.free\\.pinggy\\.link',\n",
    "        r'https://[a-zA-Z0-9\\-]+\\.pinggy\\.link',\n",
    "        r'https://[a-zA-Z0-9\\-.]+\\.pinggy\\.[a-zA-Z]+' ,\n",
    "        r'Forwarding.*?(https://[^\\s]+)',\n",
    "        r'Access.*?(https://[^\\s]+)'\n",
    "    ]\n",
    "    for p in patterns:\n",
    "        m = re.search(p, text)\n",
    "        if m:\n",
    "            return m.group(1) if m.lastindex else m.group(0)\n",
    "    return None\n",
    "\n",
    "def start_pinggy_tunnel():\n",
    "    global tunnel_proc\n",
    "    tunnel_proc = subprocess.Popen([\n",
    "        'ssh','-p','443',f'-R0:localhost:{COMFYUI_PORT}',f'-L{PINGGY_LOCAL_PORT}',\n",
    "        '-o','StrictHostKeyChecking=no','-o','ServerAliveInterval=30',PINGGY_TOKEN\n",
    "    ], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)\n",
    "\n",
    "def monitor_pinggy():\n",
    "    global tunnel_url, tunnel_ready\n",
    "    try:\n",
    "        buf=''\n",
    "        for _ in range(50):\n",
    "            line = tunnel_proc.stdout.readline()\n",
    "            if not line:\n",
    "                continue\n",
    "            print('PINGGY:', line.strip())\n",
    "            url = extract_tunnel_url(line)\n",
    "            if url:\n",
    "                tunnel_url = url\n",
    "                tunnel_ready = True\n",
    "                print('\\n🌍 Public URL:', tunnel_url)\n",
    "                break\n",
    "            buf += line\n",
    "        if not tunnel_ready:\n",
    "            url = extract_tunnel_url(buf)\n",
    "            if url:\n",
    "                tunnel_url = url\n",
    "                tunnel_ready = True\n",
    "                print('\\n🌍 Public URL:', tunnel_url)\n",
    "    except Exception as e:\n",
    "        print('Pinggy error:', e)\n",
    "\n",
    "def iframe_thread(port):\n",
    "    print('⏳ Waiting ComfyUI...')\n",
    "    while True:\n",
    "        time.sleep(0.5)\n",
    "        s = socket.socket()\n",
    "        if s.connect_ex(('127.0.0.1',port)) == 0:\n",
    "            break\n",
    "        s.close()\n",
    "    print('✔ ComfyUI ready → creating tunnel...')\n",
    "    start_pinggy_tunnel()\n",
    "    threading.Thread(target=monitor_pinggy, daemon=True).start()\n",
    "\n",
    "threading.Thread(target=iframe_thread, daemon=True, args=(COMFYUI_PORT,)).start()\n",
    "\n",
    "!pip install -r /content/ComfyUI/requirements.txt\n",
    "!wget https://huggingface.co/banhkeomath1/and/resolve/main/ComfyUI_00005_.png -P /content/ComfyUI/input\n",
    "!pip install watchdog vtracer torchsde replicate\n",
    "!pip install sageattention==2.2.0 --no-build-isolation\n",
    "\n",
    "play_audio(audio_path)\n",
    "\n",
    "print('\\n🎉 INSTALL COMPLETED – READY TO RUN COMFYUI!')\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": { "display_name": "Python 3", "language": "python", "name": "python3" },
  "language_info": { "name": "python", "version": "3.10" }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
